In [1]:
from premise import *
from datapackage import Package
import bw2io as bw

In [2]:
import bw2data
bw2data.projects.set_current("ei310")

In [3]:
clear_inventory_cache()

Inventory cache cleared!


In [4]:
ndb = NewDatabase(
    scenarios=[
        #{"model":"remind", "pathway":"SSP2-Base", "year":2050},
        #{"model":"remind", "pathway":"SSP2-Base", "year":2010},
        #{"model":"image", "pathway":"SSP2-Base", "year":2020},
        #{"model":"remind", "pathway":"SSP2-Base", "year":2050},
        {"model":"remind", "pathway":"SSP2-PkBudg1150", "year":2050},
        #{"model":"image", "pathway":"SSP2-RCP26", "year":2040},
        #{"model":"image", "pathway":"SSP2-RCP26", "year":2060},
        #{"model":"image", "pathway":"SSP2-RCP19", "year":2050},
        #{"model":"image", "pathway":"SSP2-Base", "year":2050},
    ],
    source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.10", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    biosphere_name="ecoinvent-3.10-biosphere"
)

premise v.(2, 1, 1, 'dev4')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

In [5]:
ndb.update("battery")

Processing scenarios: 100%|█████████████| 1/1 [00:09<00:00,  9.96s/it]

Done!



In [6]:
ndb.generate_change_report()

Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [6]:
ndb.write_db_to_brightway("battery 3")

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:25


Title: Writing activities to SQLite3 database:
  Started: 07/14/2024 20:32:00
  Finished: 07/14/2024 20:32:25
  Total time elapsed: 00:00:25
  CPU %: 92.00
  Memory %: 13.95
Created database: battery 3
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [6]:
ndb.generate_change_report()

Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [6]:
from premise.utils import load_database
ndb.scenarios[0] = load_database(ndb.scenarios[0])

for ds in ndb.scenarios[0]["database"]:
    if ds["name"] == "heavy duty truck, battery electric, NMC-622 battery, 40t gross weight, long haul":
        for e in ds["exchanges"]:
            print(e["name"], e["amount"])

gearbox, for lorry 201.6
transmission, for lorry 291.2
treatment of used lorry, 40 metric ton -1
market for power distribution unit, for electric passenger car 4
assembly operation, for lorry 29223.50287857302
market for converter, for electric passenger car 25
market for inverter, for electric passenger car 41.19999946653843
suspension, for lorry 3440
power electronics, for lorry 265
tires and wheels, for lorry 1422
market for battery management system, for Li-ion battery 125.73132358067981
glider lightweighting 268.7599962415552
cabin, for lorry 1153
other components, for electric lorry 1059
maintenance, lorry 40 metric ton 1
frame, blanks and saddle, for lorry 5539
market for electric motor, electric passenger car 600
market for used Li-ion battery -314.32832768712774
retarder, for lorry 67.2
market for battery cell, Li-ion, NMC622 188.597004106448
heavy duty truck, battery electric, NMC-622 battery, 40t gross weight, long haul 1


In [5]:
ndb.write_db_to_brightway()

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:36


Title: Writing activities to SQLite3 database:
  Started: 07/13/2024 20:50:02
  Finished: 07/13/2024 20:50:39
  Total time elapsed: 00:00:36
  CPU %: 90.50
  Memory %: 11.02
Created database: ei_cutoff_3.10_remind_SSP2-PkBudg1150_2050 2024-07-13
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [8]:
ndb.scenarios[0]["iam data"].carbon_capture_rate.sel(variables="cement", year=2050)

<xarray.DataArray 'value' (region: 13)> Size: 104B
array([0.24319538, 0.11075869, 0.37354394, 0.1023092 , 0.41335137,
       0.10468423, 0.19437585, 0.19803933, 0.10542634, 0.1051091 ,
       0.09484875, 0.39999575, 0.14865214])
Coordinates:
  * region     (region) object 104B 'CAZ' 'CHA' 'EUR' ... 'SSA' 'USA' 'World'
    year       int64 8B 2050
    variables  <U6 24B 'cement'

In [26]:
import csv

res = [
]

fp = "/Users/romain/GitHub/premise/premise/data/utils/export/simapro_categories_new.csv"
with open(fp, encoding="latin-1") as file:
    csv_reader = csv.reader(file, delimiter=",")
    next(csv_reader)

    for row in csv_reader:
        product = row[0].split("{")[0].strip()
        activity = row[0].split("|")[1].strip()
        res.append(
            [activity, f"{product[0].lower()+product[1:]}", row[1], row[2]]
        )

In [28]:
import pandas as pd
df=pd.DataFrame(res, columns=["name", "product", "category", "sub_category"])
df.head()

,name,product,category,sub_category
0,"market for barley grain, feed","barley grain, feed",material,Agricultural\Animal feed\Market
1,"market for barley grain, feed, organic","barley grain, feed, organic",material,Agricultural\Animal feed\Market
2,"market for barley grain, feed, Swiss integrate...","barley grain, feed, Swiss integrated production",material,Agricultural\Animal feed\Market
3,market for cottonseed meal,cottonseed meal,material,Agricultural\Animal feed\Market
4,market for Distiller's Dried Grains with Solubles,distiller's Dried Grains with Solubles,material,Agricultural\Animal feed\Market


In [30]:
df.to_csv("simapro_categories_new2.csv", index=False)

In [1]:
from premise.export import get_simapro_category_of_exchange
d = get_simapro_category_of_exchange()

In [2]:
d

{'market for barley grain, feed': {'name': 'market for barley grain, feed',
  'product': 'barley grain, feed',
  'category': 'material',
  'sub_category': 'Agricultural\\Animal feed\\Market'},
 'market for barley grain, feed, organic': {'name': 'market for barley grain, feed, organic',
  'product': 'barley grain, feed, organic',
  'category': 'material',
  'sub_category': 'Agricultural\\Animal feed\\Market'},
 'market for barley grain, feed, swiss integrated production': {'name': 'market for barley grain, feed, Swiss integrated production',
  'product': 'barley grain, feed, Swiss integrated production',
  'category': 'material',
  'sub_category': 'Agricultural\\Animal feed\\Market'},
 'market for cottonseed meal': {'name': 'market for cottonseed meal',
  'product': 'cottonseed meal',
  'category': 'material',
  'sub_category': 'Agricultural\\Animal feed\\Market'},
 "market for distiller's dried grains with solubles": {'name': "market for Distiller's Dried Grains with Solubles",
  'prod

In [7]:
txt = "hard coal {IN}| hard coal, import from Australia | Cut-off, U"

In [9]:
d[txt.lower()]

{'name': 'Hard coal {IN}| hard coal, import from Australia | Cut-off, U',
 'category': 'material',
 'sub_category': 'Fuels\\Coal\\Market\\Import'}

In [6]:
txt = "transport, freight, lorry >32 metric ton, EURO1 {ZA}| transport, freight, lorry >32 metric ton, EURO1 | Cut-off, U".lower()
txt

'transport, freight, lorry >32 metric ton, euro1 {za}| transport, freight, lorry >32 metric ton, euro1 | cut-off, u'

In [12]:
txt in d

True

In [13]:
d[txt]

{'name': 'Transport, freight, lorry >32 metric ton, EURO1 {ZA}| transport, freight, lorry >32 metric ton, EURO1 | Cut-off, U',
 'category': 'transport',
 'sub_category': 'Road\\Transformation'}

In [8]:
for x in u:
    print(x)

residues, MSWI, municipal solid waste {FR}| treatment of residues, MSWI, municipal solid waste, residual material landfill | Cut-off, U
natural gas, high pressure {RER}| evaporation of natural gas, import from NO | Cut-off, U
residues, MSWI[F], waste plastic, mixture {CH}| market for residues, MSWI[F], waste plastic, mixture | Cut-off, U
electricity, low voltage {US-TRE}| market for electricity, low voltage | Cut-off, U
natural gas, liquefied {KR}| natural gas, liquefied, import from ID | Cut-off, U
bottom ash, MSWI, waste packaging paper {GLO}| market for bottom ash, MSWI, waste packaging paper | Cut-off, U
natural gas, high pressure {GB}| natural gas, high pressure, import from PE | Cut-off, U
sewage sludge, 70% water, WWT-SLF, refinery sludge {RoW}| treatment of sewage sludge, 70% water, WWT-SLF, refinery sludge, municipal incineration | Cut-off, U
ethylene glycol {RER}| market for ethylene glycol | Cut-off, U
leachate, SLF, hard coal ash {GLO}| market for leachate, SLF, hard coal a

In [7]:
len(y)

17386

In [8]:
len(u)

8308

In [5]:
ndb.write_db_to_brightway("cement")

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:35


Title: Writing activities to SQLite3 database:
  Started: 06/08/2024 12:33:49
  Finished: 06/08/2024 12:34:25
  Total time elapsed: 00:00:35
  CPU %: 91.90
  Memory %: 15.98
Created database: cement
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [9]:
from multiprocessing import Process, current_process
import logging
import os

def configure_logging(process_id):
    """Configures logging for a process with a unique file per process."""
    log_filename = f"process_{process_id}.log"
    
    logger = logging.getLogger(f'worker_{process_id}')
    logger.setLevel(logging.DEBUG)
    file_handler = logging.FileHandler(log_filename, mode='a')
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    logger.propagate = False
    return logger

def main():
    processes = []
    for _ in range(5):
        # Start each process with a unique configuration based on the process ID.
        process = Process(target=lambda: configure_logging(current_process().pid).info(f"This is an info message from process {current_process().pid}."))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

if __name__ == "__main__":
    main()


AttributeError: Can't pickle local object 'main.<locals>.<lambda>'

In [ ]:
import bw2data
bw2data.projects.set_current("new4")

In [ ]:
preserve = [
    "biosphere3",
    "ecoinvent 3.9.1 cutoff",
    "ecoinvent 3.9.1 consequential",
    "ecoinvent EN15804 3.9.1",
    "ecoinvent 3.6 cutoff",
    "ecoinvent 3.7 cutoff",
    "ecoinvent 3.8 cutoff",
    
]

for db in list(bw2data.databases):
    if db not in preserve:
        del bw2data.databases[db]

In [ ]:
ndb.write_db_to_brightway()

In [ ]:
import numpy as np
import pandas as pd
df = pd.DataFrame(
np.hstack(
    [ndb.scenarios[0]["iam data"].electricity_markets.sel(region="EUR"),
    ndb.scenarios[1]["iam data"].electricity_markets.sel(region="EUR"),
    ndb.scenarios[2]["iam data"].electricity_markets.sel(region="EUR"),
    ndb.scenarios[3]["iam data"].electricity_markets.sel(region="EUR"),]
), columns=[2020, 2030, 2040, 2050],)

In [ ]:
df.index = ['Geothermal', 'Biomass CHP', 'Gas CHP', 'Wind Offshore', 'Coal PC CCS',
 'Coal CHP','Gas CC CCS','Gas ST','Oil ST','Coal IGCC','Gas CC','Coal PC',
 'Biomass IGCC CCS','Hydro','Storage, Hydrogen','Coal IGCC CCS','Nuclear',
 'Wind Onshore','Biomass IGCC','Solar PV Centralized','Solar CSP']

In [ ]:
df

In [ ]:
ndb.generate_scenario_report()

In [ ]:
ndb.

In [ ]:
import json
import os
import glob
from pprint import pprint

#fp="/Users/romain/Documents/premise_test/flows"
fp="/Users/romain/Documents/test_olca_39_4/flows"

l = []
#Load the images from images folder.
for f in glob.glob(f'{fp}/*.json'):
    with open(f) as fd:
     json_data = json.load(fd)
     l.append([json_data["name"], json_data["category"], json_data["@id"]])
    

In [ ]:
import pandas as pd
pd.DataFrame(l, columns=["Source flow", "Source category", "id"]).to_excel("mapping_source_2.xlsx")

In [ ]:
import csv


fp_in = "/Users/romain/Documents/flow_mapping_premise4_.csv"
fp_out = "/Users/romain/Documents/flow_mapping_premise5.csv"

l = []
with open(fp_in) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for r in csv_reader:
        l.append(r)
    
with open(fp_out, 'w', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerows(l)

In [ ]:
import csv


fp_in = "/Users/romain/Downloads/sp_flow_import_map.csv"
fp_out = "/Users/romain/Downloads/sp_flow_import_map2.csv"

l = []
with open(fp_in) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for r in csv_reader:
        l.append(r)
    
with open(fp_out, 'w', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerows(l)

In [ ]:
ndb.update_all()

In [ ]:
scenarios = [
    {"model": "remind", "pathway":"SSP1-Base", "year": 2050},
    {"model": "remind", "pathway":"SSP1-NDC", "year": 2050},
    #{"model": "remind", "pathway":"SSP1-NPi", "year": 2050},
    #{"model": "remind", "pathway":"SSP1-PkBudg1150", "year": 2050},
    #{"model": "remind", "pathway":"SSP1-PkBudg500", "year": 2050},
    #{"model": "remind", "pathway":"SSP5-Base", "year": 2050},
    #{"model": "remind", "pathway":"SSP5-NDC", "year": 2050},
    #{"model": "remind", "pathway":"SSP5-NPi", "year": 2050},
    #{"model": "remind", "pathway":"SSP5-PkBudg1150", "year": 2050},
    #{"model": "remind", "pathway":"SSP5-PkBudg500", "year": 2050},
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ecoinvent 3.9 cutoff",
        source_version="3.9",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
)


In [ ]:
ndb.update_all()

In [ ]:
ndb.write_superstructure_db_to_brightway("test")

In [ ]:
bw.BW2Package.export_obj(bw.Database("test"))

In [ ]:
import numpy as np
for dataset in bw.Database("test"):
    #for exc in dataset.exchanges():
    #    if not isinstance(exc["amount"], float):
    #        print(dataset["name"], type(exc["amount"]))

    #    if isinstance(exc["amount"], np.float64):
    #        print(dataset["name"], type(exc["amount"]))

    for k, v in dataset.items():
        if isinstance(v, dict):
            for i, j in v.items():
                if isinstance(j, np.float64):
                    #continue
                    print(i, j)
                    
                if not type(j) in [str, float, int, tuple, list, dict, bool, None]:
                    print("1", dataset["name"], i, j, type(j), float(j))

    for e in dataset.exchanges():
        for k, v in e.items():
            if isinstance(v, np.float64):
                #continue
                print(k, v)
                
            if not type(v) in [str, float, int, tuple, list, dict, bool, None]:
                print("2", dataset["name"], k, v, type(v), float(v))

In [ ]:
ndb.write_datapackage()

In [ ]:
from unfold import Unfold
import brightway2 as bw
bw.projects.set_current("ei39")

In [ ]:
u = Unfold("export/datapackage/datapackage_2023-04-20.zip")

In [ ]:
u.unfold(superstructure=True, name="my_super_db")

In [ ]:
dict(enumerate(["a", "b", "c"]))

In [ ]:
ndb.write_db_to_brightway("test")

In [ ]:
variables = ndb.scenarios[0]["iam data"].fuel_markets.variables.values
ndb.scenarios[0]["iam data"].fuel_markets.sel(region="EUR", variables=[v for v in variables if "diesel" in v], year=2050).to_dataframe("val")

In [ ]:
ndb.scenarios[0]["iam data"].gains_data_EU

In [ ]:
ndb.write_db_to_brightway("test")

In [ ]:
ndb.write_datapackage()

In [ ]:
from unfold import Unfold

In [ ]:
u = Unfold("/Users/romain/Downloads/premise_remind_ssp2_base.zip")

In [ ]:
u.unfold(scenarios=[98, 99])

In [ ]:
from unfold import Fold

In [ ]:
f = Fold()

In [ ]:
f.fold()

In [ ]:
import premise

In [ ]:
premise.__file__

In [ ]:
from premise import clear_cache

In [ ]:
clear_cache()